In [1]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import numpy as np
from math import *
from shapely.geometry import Point, Polygon
import geopandas as gpd
import sys
print(sys.path)
import math
from tqdm import tqdm
from pandas import DataFrame
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from kneed import KneeLocator
from sklearn.neighbors import NearestNeighbors, KDTree
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
from threadpoolctl import threadpool_limits

import pymongo
import json
import pandas as pd
from datetime import datetime, timedelta
from pandas import DataFrame
import matplotlib.pyplot as plt
from datetime import time
from tqdm import tqdm

from math import *
from shapely.geometry import Point, Polygon
import geopandas as gpd
import sys
print(sys.path)
import math
from pandas import DataFrame
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from kneed import KneeLocator
from sklearn.neighbors import KDTree
from sklearn.cluster import DBSCAN

/opt/anaconda3/envs/finding_insights_data/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


['/Users/ohidabinteamin/Documents/Stress Prediction Project Three Datasets/StudentLife/week 01/Location', '/opt/anaconda3/envs/finding_insights_data/lib/python39.zip', '/opt/anaconda3/envs/finding_insights_data/lib/python3.9', '/opt/anaconda3/envs/finding_insights_data/lib/python3.9/lib-dynload', '', '/opt/anaconda3/envs/finding_insights_data/lib/python3.9/site-packages']
['/Users/ohidabinteamin/Documents/Stress Prediction Project Three Datasets/StudentLife/week 01/Location', '/opt/anaconda3/envs/finding_insights_data/lib/python39.zip', '/opt/anaconda3/envs/finding_insights_data/lib/python3.9', '/opt/anaconda3/envs/finding_insights_data/lib/python3.9/lib-dynload', '', '/opt/anaconda3/envs/finding_insights_data/lib/python3.9/site-packages']


In [2]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["studentlife"]

In [3]:
student_uids = ['u00', 'u01', 'u02', 'u03', 'u04', 'u05', 'u07', 'u08', 'u09', 'u10', 'u12', 'u13', 'u14', 'u15', 'u16',
                'u17', 'u18', 'u19', 'u20', 'u22', 'u23', 'u24', 'u25', 'u27', 'u30', 'u31', 'u32', 'u33', 'u34', 'u35',
                'u36', 'u39', 'u41', 'u42', 'u43', 'u44', 'u45', 'u46', 'u47', 'u49', 'u50', 'u51', 'u52', 'u53', 'u54',
                'u56', 'u57', 'u58', 'u59']
print(len(student_uids))

49


In [4]:
df_gps = mydb.gps
df_gps = DataFrame(list(df_gps.find()))
df_gps = df_gps.drop("_id", axis=1)

df_gps = df_gps[["uid", "timestamp", "latitude", "longitude"]]

In [5]:
df_gps.head(3)

,uid,timestamp,latitude,longitude
0,u45,1364410654,43.7066051,-72.2870424
1,u45,1364411866,43.7065982,-72.2870054
2,u45,1364852743,43.706614,-72.2870392


In [6]:
# morning (6 am - 12 pm) 
# afternoon (12 pm - 6 pm) 
# evening (6 pm - 12 am) 
# night (12 am - 6 am) 

In [7]:
df_gps['timestamp'] = pd.to_datetime(df_gps['timestamp'], unit='s')
df_gps['date'] = df_gps['timestamp'].dt.strftime('%Y-%m-%d')
df_gps['timestamp'] = df_gps['timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [8]:
morning_start_time = time(6, 0, 0)     # 6:00 AM
afternoon_start_time = time(12, 0, 0)  # 12:00 PM
evening_start_time = time(18, 0, 0)    # 6:00 PM
night_start_time = time(0, 0, 0)       # 12:00 AM (midnight)

df_gps['timestamp'] = pd.to_datetime(df_gps['timestamp'])
df_gps['time'] = df_gps['timestamp'].dt.time

def categorize_time_interval(timestamp):
    current_time = timestamp.time()
    
    if morning_start_time <= current_time < afternoon_start_time:
        return "morning"
    elif afternoon_start_time <= current_time < evening_start_time:
        return "afternoon"
    elif evening_start_time <= current_time <= time(23, 59, 59):
        return "evening"
    else:
        return "night"

df_gps['time_interval'] = df_gps['timestamp'].apply(categorize_time_interval)
df_gps = df_gps.drop("time", axis=1)
df_gps.head(3)

,uid,timestamp,latitude,longitude,date,time_interval
0,u45,2013-03-27 18:57:34,43.7066051,-72.2870424,2013-03-27,evening
1,u45,2013-03-27 19:17:46,43.7065982,-72.2870054,2013-03-27,evening
2,u45,2013-04-01 21:45:43,43.706614,-72.2870392,2013-04-01,evening


In [9]:
print(len(df_gps['uid'].unique()))
df_gps['time_interval'].value_counts()

49


morning      51198
night        51182
afternoon    50305
evening      50192
Name: time_interval, dtype: int64

In [10]:
df_gps.isnull().sum()
df_gps.to_csv('StudentLife_location_data.csv')

In [11]:
# number of different locations visited
# variance in time spent at different locations 

In [12]:
# def count_places_visited(df, eps=0.5, min_samples=3):
#     results = []
#     unique_dates = df['date'].unique()

#     for date in unique_dates:
#         places_count = {
#             'number_places_morning': 0,
#             'number_places_afternoon': 0,
#             'number_places_evening': 0,
#             'number_places_night': 0,
#             'time_spent_morning': [],
#             'time_spent_afternoon': [],
#             'time_spent_evening': [],
#             'time_spent_night': [],
#             'variance_time_spent_morning': 0,
#             'variance_time_spent_afternoon': 0,
#             'variance_time_spent_evening': 0,
#             'variance_time_spent_night': 0
#         }
        
#         for time_interval in ['morning', 'afternoon', 'evening', 'night']:
#             subset = df[(df['date'] == date) & 
#                         (df['time_interval'] == time_interval)].copy()
            
#             if not subset.empty:
#                 coords = subset[['latitude', 'longitude']].values
#                 db = DBSCAN(eps=eps, min_samples=min_samples).fit(coords)
#                 labels = db.labels_
#                 unique_labels = set(labels)
                
#                 num_places = len(unique_labels) - (1 if -1 in unique_labels else 0)
#                 places_count[f'number_places_{time_interval}'] = num_places

#                 # time spent at each location
#                 for label in unique_labels:
#                     if label != -1:
#                         location_subset = subset[labels == label].sort_values(by='timestamp')
#                         time_spent = 0
                        
#                         for i in range(len(location_subset) - 1):
#                             if labels[i] == labels[i + 1]:
#                                 time_spent += (location_subset['timestamp'].iloc[i + 1] - 
#                                                location_subset['timestamp'].iloc[i]).total_seconds()
#                             else:
#                                 time_spent += 1

#                         time_spent /= 3600  # convert to hours
#                         places_count[f'time_spent_{time_interval}'].append(time_spent)

#                 # variance in time spent at different locations
#                 if places_count[f'time_spent_{time_interval}']:
#                     places_count[f'variance_time_spent_{time_interval}'] = np.var(places_count[f'time_spent_{time_interval}'])
#             else:
#                 places_count[f'number_places_{time_interval}'] = 0
#                 places_count[f'variance_time_spent_{time_interval}'] = 0
        
#         results.append({
#             'uid': df['uid'].iloc[0],
#             'date': date,
#             **places_count
#         })
        
#     results_df = pd.DataFrame(results)
#     del results_df['time_spent_morning']
#     del results_df['time_spent_afternoon']
#     del results_df['time_spent_evening']
#     del results_df['time_spent_night']
    
#     return results_df

In [13]:
# result_1 = DataFrame()

# for uid in tqdm(student_uids):
#     print(uid)
#     df_ = df_gps[df_gps['uid'] == uid]
#     df__ = count_places_visited(df_)
#     frames = [result_1, df__]
#     result_1 = pd.concat(frames, ignore_index=True)